In [1]:
!pip install datasets
!pip install transformers
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: h

In [2]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import numpy as np
from huggingface_hub import login
import torch

In [3]:
# Token stored in secrets
login()

In [6]:
dataset = load_dataset("newsmediabias/news-bias-full-data")

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
for i in range (10):
  print (dataset['train'][i])

{'text': 'Where will this leave the kids??? That my friends, is the saddest part of all.  ', 'dimension': 'Ageism and sentiment', 'biased_words': '[]', 'aspect': 'Transphobia', 'label': 'Slightly Biased', 'sentiment': 'Negative', 'toxic': 0.0, 'identity_mention': 'NO'}
{'text': ' I had a feeling of impending doom &amp; then hear about this. NO GOOD. NO GOOD.', 'dimension': 'Ageism and sentiment', 'biased_words': "['impending', 'doom']", 'aspect': 'Climate change', 'label': 'Highly Biased', 'sentiment': 'Negative', 'toxic': 1.0, 'identity_mention': 'NO'}
{'text': 'I no feel good .. ', 'dimension': 'Ageism and sentiment', 'biased_words': '[]', 'aspect': 'Body-shaming', 'label': 'Neutral', 'sentiment': 'Positive', 'toxic': 0.0, 'identity_mention': 'NO'}
{'text': "Back in Hotlanta for a about a week, then off to Texas for a day, then off to Vegas to win us some money...or lose some   I'm exhausted!", 'dimension': 'Ageism and sentiment', 'biased_words': '[]', 'aspect': 'Lifestyle', 'label':

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [23]:
train_df = dataset["train"].to_pandas()
test_df = dataset["test"].to_pandas()

In [27]:
train_df.head()

,text,dimension,biased_words,aspect,label,sentiment,toxic,identity_mention
0,Where will this leave the kids??? That my frie...,Ageism and sentiment,[],Transphobia,Slightly Biased,Negative,0.0,NO
1,I had a feeling of impending doom &amp; then ...,Ageism and sentiment,"['impending', 'doom']",Climate change,Highly Biased,Negative,1.0,NO
2,I no feel good ..,Ageism and sentiment,[],Body-shaming,Neutral,Positive,0.0,NO
3,"Back in Hotlanta for a about a week, then off ...",Ageism and sentiment,[],Lifestyle,Slightly Biased,Negative,0.0,NO
4,Kids toys &amp; stamping stuff cleaned up... N...,Ageism and sentiment,[],Hobby,Neutral,Neutral,1.0,NO


In [26]:
set(train_df['label'].unique())

{'Highly Biased', 'Neutral', 'Slightly Biased'}

In [88]:
def tokenize(batch):
    texts = [element["text"] for element in batch]
    # label_to_int = {'Highly Biased': 2, 'Slightly Biased': 1, 'Neutral': 0}
    label_to_tensor = {
        "1.0": torch.tensor([1, 0]),
        "0.0": torch.tensor([0, 1]),
    }
    # print (torch.cat([label_to_tensor[str(element["toxic"])] for element in batch]).shape)
    encoded_texts = tokenizer(texts, padding=True, truncation=True, max_length=256)
    return {
        "input_ids": torch.tensor(encoded_texts["input_ids"]).to(device),
        "attention_mask": torch.tensor(encoded_texts["attention_mask"]).to(device),
        "labels": torch.tensor(torch.cat([label_to_tensor[str(element["toxic"])] for element in batch]).view(-1,2)).float().to(device)
    }

train_set = torch.utils.data.Subset(dataset['train'], list(range(10000)))
test_set = torch.utils.data.Subset(dataset['test'], list(range(1000)))
train_dataloader = torch.utils.data.DataLoader(train_set, collate_fn=tokenize, batch_size=16)
test_dataloader = torch.utils.data.DataLoader(dataset['test'], collate_fn=tokenize, batch_size=16)

In [81]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss

optimizer = AdamW(model.parameters())
loss_function = CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [107]:
model.train()
num_epochs = 3

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Forward pass
        outputs = model(**batch)
        loss = loss_function(outputs.logits, batch["labels"])

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    model.eval()
    num_correct_positive = 0
    total_actual_positive = 0
    total_predicted_positive = 0
    with torch.no_grad():
        for batch in test_dataloader:
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=1)

            labels = batch["labels"]

            num_correct_positive += torch.sum((predictions == 1) & (labels == 1)[:, 0])
            total_predicted_positive += torch.sum(predictions == 1)
            total_actual_positive += torch.sum((labels == 1)[:, 0])

    precision = num_correct_positive / total_predicted_positive
    recall = num_correct_positive / total_actual_positive
    print (precision, recall)

    # Save the model checkpoint
    torch.save(model.state_dict(), f"model_epoch_{epoch}.pt")

<ipython-input-88-ee8550ed5893>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(torch.cat([label_to_tensor[str(element["toxic"])] for element in batch]).view(-1,2)).float().to(device)


tensor(0.4857, device='cuda:0') tensor(1., device='cuda:0')
tensor(0.4857, device='cuda:0') tensor(1., device='cuda:0')
tensor(0.4857, device='cuda:0') tensor(1., device='cuda:0')
